<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**


---

<p align=center>
<img src="img/fetal_health.jpg" width="40%"><br>
<i><sup>Image credits: freepik (<a href="https://br.freepik.com/vetores-gratis/gravidez-e-cenas-de-maternidade_9882829.htm">www.freepik.com</a>)</sup></i>
</p>
